In [1]:
import numpy as np
import pandas as pd

In [2]:
# Binary SVM Classifier
class BinarySVM:
    def __init__(self, C=1.0, learning_rate=0.001, n_iters=1000):
        self.C = C
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for iteration in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y[idx] * (np.dot(x_i, self.weights) - self.bias) >= 1
                if condition:
                    self.weights -= self.learning_rate * (2 * self.C * self.weights)
                else:
                    self.weights -= self.learning_rate * (2 * self.C * self.weights - np.dot(x_i, y[idx]))
                    self.bias -= self.learning_rate * y[idx]

            # Debugging: print weights and bias at every 100 iterations
            if iteration % 100 == 0:
                print(f"Iteration {iteration}: Weights: {self.weights}, Bias: {self.bias}")

    def predict(self, X):
        return np.sign(np.dot(X, self.weights) - self.bias)

In [3]:

# One-vs-One SVM Classifier
class OneVsOneSVM:
    def __init__(self, C=1.0, learning_rate=0.001, n_iters=1000):
        self.C = C
        self.learning_rate = learning_rate
        self.n_iters = n_iters
        self.classifiers = []

    def fit(self, X, y):
        unique_classes = np.unique(y)
        n_classes = len(unique_classes)
        print(f"Unique classes: {unique_classes}")  # Debugging line
        for i in range(n_classes):
            for j in range(i + 1, n_classes):
                class_i = unique_classes[i]
                class_j = unique_classes[j]

                # Filter the data for the two classes
                idx = np.where((y == class_i) | (y == class_j))
                X_filtered = X[idx]
                y_filtered = y[idx]

                # Convert class labels to +1 and -1
                y_filtered = np.where(y_filtered == class_i, 1, -1)

                # Train the binary classifier
                clf = BinarySVM(C=self.C, learning_rate=self.learning_rate, n_iters=self.n_iters)
                clf.fit(X_filtered, y_filtered)
                self.classifiers.append((clf, (class_i, class_j)))

    def predict(self, X):
        n_classes = len(np.unique(y))
        votes = np.zeros((X.shape[0], n_classes))

        for clf, class_labels in self.classifiers:
            predictions = clf.predict(X)
            print(f"Classifier {class_labels} predictions: {predictions}")  # Debugging line
            for idx, pred in enumerate(predictions):
                if pred == 1:
                    votes[idx, class_labels[0] - 1] += 1  # Adjust index for zero-based array
                else:
                    votes[idx, class_labels[1] - 1] += 1  # Adjust index for zero-based array

        # Return the class with the most votes
        final_predictions = np.argmax(votes, axis=1)
        print(f"Votes: {votes}")  # Debugging line
        print(f"Final predictions: {final_predictions}")  # Debugging line
        return final_predictions
